In [1]:
import sys
sys.path.append(r"D:\Clarifruit\cherry_stem")

from work.unet_main import *
from work.unet import unet_model_functions

from keras.callbacks import ReduceLROnPlateau,EarlyStopping

Using TensorFlow backend.


In [2]:
train_path=r'D:\Clarifruit\cherry_stem\data\raw_data\with_maskes'
dest_path=r'D:\Clarifruit\cherry_stem\data\unet_data\training'
test_path=r'D:\Clarifruit\cherry_stem\data\raw_data\images_orig'
src_path=r'D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55'
weights_name =r'unet_cherry_stem.3008-0.08.hdf5'


In [3]:
params_dict = dict(

    train_path=train_path,
    save_path=dest_path,
    x_folder_name='image',
    y_folder_name='label',
    weights_file_name='unet_cherry_stem.hdf5',

    data_gen_args=dict(rotation_range=180,
                       brightness_range=[0.2, 1.],
                       shear_range=10,
                       zoom_range=0.5,
                       horizontal_flip=True,
                       vertical_flip=True,
                       channel_shift_range=5,
                       fill_mode='nearest'),
    seed=78,


    optimizer='Adam',
    optimizer_params=dict(lr=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'],
    pretrained_weights=None,

    target_size=(256, 256),
    color_mode='rgb',
    
    batch_size=10, # my GPU cant handel any more
    
    epochs=10,
    steps_per_epoch=3000,
    valdiation_split=0.2,
    validation_steps=300,
    
    tensorboard_update_freq=1000,
    weights_update_freq = 5000,
    ontop_display_threshold=0.4)

In [ ]:
model = unet_model_functions.ClarifruitUnet(**params_dict)
keras_logs_path=model.set_model_for_train(params_dict=params_dict)

In [4]:
# load model
params_dict = unet_model_functions.ClarifruitUnet.load_model(src_path,weights_name)
model = unet_model_functions.ClarifruitUnet(**params_dict)
keras_logs_path=model.set_model_for_train(params_dict=params_dict)

2019-10-13 19:19:44,552-6d  INFO     work.unet.unet_model: <- unet model with input_size=(256, 256, 3)
2019-10-13 19:19:44,553-6d  INFO     work.unet.unet_model:  model loaded from pretraind_weights=D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\unet_cherry_stem.3008-0.08.hdf5
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.


In [5]:
print(f"tensorboard --logdir={keras_logs_path}")

tensorboard --logdir=D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\keras_logs


In [ ]:
model.fit_unet()

Epoch 1/10
 526/3000 [====>.........................] - ETA: 39:08 - loss: 0.0548 - acc: 0.7333
step 05004: loss improved from inf to 0.05043, saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\unet_cherry_stem.5004-0.05.hdf5
1053/3000 [=========>....................] - ETA: 30:59 - loss: 0.0492 - acc: 0.7316
step 10010: loss improved from 0.05043 to 0.03616, saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_18-57-55\unet_cherry_stem.10010-0.04.hdf5
1580/3000 [==============>...............] - ETA: 22:42 - loss: 0.0463 - acc: 0.7275
steps 15016: loss did not improve from 0.03616
2107/3000 [====================>.........] - ETA: 14:18 - loss: 0.0443 - acc: 0.7311
steps 20022: loss did not improve from 0.03616
2634/3000 [=========================>....] - ETA: 5:57 - loss: 0.0429 - acc: 0.7326
steps 25028: loss did not improve from 0.03616
2846/3000 [===========================>..] - ETA: 2:32 - loss: 0.0425 - acc: 0.7325

In [5]:
model.prediction(test_path,dest_path)


2019-10-13 09:48:57,533-6d  INFO     work.unet.unet_model_functions: prediction on files from D:\Clarifruit\cherry_stem\data\raw_data\images_orig
2019-10-13 09:48:57,535-6d  INFO     work.unet.unet_model_functions: saving predictions to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-13_03-58-48\raw_pred


'D:\\Clarifruit\\cherry_stem\\data\\unet_data\\training\\2019-10-13_03-58-48\\raw_pred'